<a href="https://colab.research.google.com/github/waqasde/Batch52/blob/main/Library_WA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from typing import List, Optional, Union
import os

# Base class for User
class User:
    def __init__(self, user_id: str, name: str, email: str):
        self._user_id = user_id
        self._name = name
        self._email = email

    def get_user_info(self) -> dict:
        return {"user_id": self._user_id, "name": self._name, "email": self._email}

# Librarian class (inherits from User)
class Librarian(User):
    def __init__(self, user_id: str, name: str, email: str):
        super().__init__(user_id, name, email)

# Member class (inherits from User)
class Member(User):
    def __init__(self, user_id: str, name: str, email: str):
        super().__init__(user_id, name, email)
        self.borrowed_books: List[str] = []

    def borrow_book(self, book_id: str, library_manager):
        library_manager.borrow_book(self, book_id)

    def return_book(self, book_id: str, library_manager):
        library_manager.return_book(self, book_id)

# Book class
class Book:
    def __init__(self, book_id: str, title: str, author: str, available: bool = True):
        self._book_id = book_id
        self._title = title
        self._author = author
        self._available = available

    def display_info(self) -> dict:
        return {
            "book_id": self._book_id,
            "title": self._title,
            "author": self._author,
            "available": self._available
        }

    def is_available(self) -> bool:
        return self._available

    def set_availability(self, status: bool):
        self._available = status

# Library Manager class to handle book/user CRUD and transactions
class LibraryManager:
    books_file = "books.txt"
    users_file = "users.txt"

    def __init__(self):
        self.books = self._load_books()
        self.users = self._load_users()

    def _load_books(self) -> List[Book]:
        books = []
        if os.path.exists(self.books_file):
            with open(self.books_file, "r") as f:
                for line in f.readlines():
                    book_id, title, author, available = line.strip().split(",")
                    books.append(Book(book_id, title, author, available == "True"))
        return books

    def _load_users(self) -> List[Union[Librarian, Member]]:
        users = []
        if os.path.exists(self.users_file):
            with open(self.users_file, "r") as f:
                for line in f.readlines():
                    user_id, name, email, user_type = line.strip().split(",")
                    if user_type == "Librarian":
                        users.append(Librarian(user_id, name, email))
                    elif user_type == "Member":
                        users.append(Member(user_id, name, email))
        return users

    def save_books(self):
        with open(self.books_file, "w") as f:
            for book in self.books:
                book_info = book.display_info()
                f.write(f"{book_info['book_id']},{book_info['title']},{book_info['author']},{book_info['available']}\n")

    def save_users(self):
        with open(self.users_file, "w") as f:
            for user in self.users:
                user_info = user.get_user_info()
                user_type = "Librarian" if isinstance(user, Librarian) else "Member"
                f.write(f"{user_info['user_id']},{user_info['name']},{user_info['email']},{user_type}\n")

    def add_book(self, book: Book):
        self.books.append(book)
        self.save_books()

    def update_book(self, book_id: str, title: Optional[str] = None, author: Optional[str] = None, available: Optional[bool] = None):
        for book in self.books:
            if book._book_id == book_id:
                if title:
                    book._title = title
                if author:
                    book._author = author
                if available is not None:
                    book.set_availability(available)
                self.save_books()
                return
        print(f"Book with ID {book_id} not found.")

    def delete_book(self, book_id: str):
        self.books = [book for book in self.books if book._book_id != book_id]
        self.save_books()

    def add_user(self, user: Union[Librarian, Member]):
        self.users.append(user)
        self.save_users()

    def borrow_book(self, member: Member, book_id: str):
        for book in self.books:
            if book._book_id == book_id and book.is_available():
                book.set_availability(False)
                member.borrowed_books.append(book_id)
                self.save_books()
                print(f"{member.get_user_info()['name']} borrowed {book.display_info()['title']}.")
                return
        print(f"Book with ID {book_id} is not available.")

    def return_book(self, member: Member, book_id: str):
        if book_id in member.borrowed_books:
            for book in self.books:
                if book._book_id == book_id:
                    book.set_availability(True)
                    member.borrowed_books.remove(book_id)
                    self.save_books()
                    print(f"{member.get_user_info()['name']} returned {book.display_info()['title']}.")
                    return
        print(f"Book with ID {book_id} not found in borrowed list.")

# Example usage
if __name__ == "__main__":
    manager = LibraryManager()

    # Create a librarian and member
    librarian = Librarian("K1", "Waqas Ahmad", "waqasde@gmail.com")
    member = Member("M4", "Ali", "Ali@yahoo.com")

    # Add users to the system
    manager.add_user(librarian)
    manager.add_user(member)

    # Librarian adds a book
    book = Book("B1", "The Great Journey", "Agathe Demois")
    manager.add_book(book)

    # Member borrows a book
    member.borrow_book("B1", manager)

    # Member returns a book
    member.return_book("B1", manager)

    # List all books
    for book in manager.books:
        print(book.display_info())


Ali borrowed The Great Gatsby.
Ali returned The Great Gatsby.
{'book_id': 'B1', 'title': 'The Great Gatsby', 'author': 'F. Scott Fitzgerald', 'available': True}
{'book_id': 'B1', 'title': 'The Great Journey', 'author': 'Agathe Demois', 'available': True}
